In [1]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split


2023-02-28 21:33:21.615003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 21:33:21.730781: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 21:33:21.730798: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-28 21:33:22.397453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
df = pd.read_csv('./data/processed_data.csv')
df.shape


(18471, 69)

In [3]:
# Final labels
labels = ['patientState_good', 'patientState_caution', 'patientState_danger']
# Section labels
section_labels = {
    'fever': ['feverState_good', 'feverState_caution', 'feverState_danger'],
    'medication': ['medicationState_good', 'medicationState_caution', 'medicationState_danger'],
    'hydration': ['hydrationState_good', 'hydrationState_caution', 'hydrationState_danger'],
    'respiration': ['respirationState_good', 'respirationState_caution', 'respirationState_danger'],
    'skin': ['skinState_good', 'skinState_caution', 'skinState_danger'],
    'pulse': ['pulseState_good', 'pulseState_caution', 'pulseState_danger'],
    'general': ['generalState_good', 'generalState_caution', 'generalState_danger'],
}
# Section labels in an array flattened
section_labels_arr = [item for val in section_labels.values() for item in val]
# All labels
all_labels = [*labels, *section_labels_arr]
len(all_labels)


24

In [4]:
df[section_labels['general']].value_counts()


generalState_good  generalState_caution  generalState_danger
1.0                0.0                   0.0                    10142
0.0                1.0                   0.0                     7976
                   0.0                   1.0                      353
dtype: int64

In [5]:
# Separating to testing and training

# Prepare the data: You should split your data into training and test sets.
# The training set will be used to train the model and the test set will be used
# to evaluate the model's performance.

_x, x_test, _y, y_test = train_test_split(
    df.drop(columns=all_labels).to_numpy(),
    df[labels].to_numpy(),
    test_size=0.2,
    random_state=42
)
x_train, x_val, y_train, y_val = train_test_split(
    _x,
    _y,
    test_size=0.25,
    train_size=0.75,
    random_state=42
)
x_train.shape


(11082, 45)

In [6]:
str(list(x_test[1]))


'[15.6173706445342, 1.0, 38.3, 0.0, 0.0, 0.0, 1.0, 1.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 68.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]'

In [8]:
# Preprocess the data: Data preprocessing is an important step, which includes
# cleaning and transforming the data. You should normalize the data,
# one-hot encode categorical variables, and split the data into features and labels.

# TODO: Does KNNImputer create a correlation between training and test split?
# If yes do ->
# TODO: move KNNImputer here
# TODO: move every preprocessing step that would create a connection between train and test split


In [7]:
from tensorflow import keras


In [8]:
# Create a baseline model to measure performance
b_model = keras.Sequential([
    keras.layers.Dense(units=45, activation='tanh', input_shape=(45,)),
    keras.layers.Dense(units=3, activation='softmax'),
])
b_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
b_model.build()
b_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 45)                2070      
                                                                 
 dense_1 (Dense)             (None, 3)                 138       
                                                                 
Total params: 2,208
Trainable params: 2,208
Non-trainable params: 0
_________________________________________________________________


2023-02-28 21:33:34.369753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 21:33:34.370121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 21:33:34.370174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-28 21:33:34.370220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-28 21:33:34.370266: W tensorflow/c

In [9]:
# Train baseline model
NUM_EPOCHS = 20

# Early stopping set after 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

b_model.fit(
    x_train,
    y_train,
    epochs=NUM_EPOCHS,
    validation_data=(x_val, y_val),
    callbacks=[stop_early],
    verbose=2
)

Epoch 1/20
347/347 - 1s - loss: 0.7627 - accuracy: 0.6428 - val_loss: 0.6594 - val_accuracy: 0.6984 - 1s/epoch - 3ms/step
Epoch 2/20
347/347 - 0s - loss: 0.6201 - accuracy: 0.7361 - val_loss: 0.5683 - val_accuracy: 0.7675 - 403ms/epoch - 1ms/step
Epoch 3/20
347/347 - 0s - loss: 0.5160 - accuracy: 0.7933 - val_loss: 0.4769 - val_accuracy: 0.8045 - 400ms/epoch - 1ms/step
Epoch 4/20
347/347 - 0s - loss: 0.4507 - accuracy: 0.8223 - val_loss: 0.4528 - val_accuracy: 0.8211 - 391ms/epoch - 1ms/step
Epoch 5/20
347/347 - 0s - loss: 0.4127 - accuracy: 0.8404 - val_loss: 0.4048 - val_accuracy: 0.8481 - 381ms/epoch - 1ms/step
Epoch 6/20
347/347 - 0s - loss: 0.3938 - accuracy: 0.8502 - val_loss: 0.3862 - val_accuracy: 0.8522 - 380ms/epoch - 1ms/step
Epoch 7/20
347/347 - 0s - loss: 0.3766 - accuracy: 0.8546 - val_loss: 0.3714 - val_accuracy: 0.8533 - 393ms/epoch - 1ms/step
Epoch 8/20
347/347 - 0s - loss: 0.3623 - accuracy: 0.8566 - val_loss: 0.3866 - val_accuracy: 0.8389 - 409ms/epoch - 1ms/step
Epo

In [10]:
def evaluate_model(model, x_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(x_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

# Evaluate model on test set and add results to dataframe
results = evaluate_model(b_model, x_test, y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

116/116 [==============================] - 0s 1ms/step - loss: 0.3018 - accuracy: 0.8844


,loss,accuracy
Baseline,0.301824,0.884438


In [12]:
# Define the model: TensorFlow provides a high-level API for building and
# training neural network models. You should choose the appropriate model
# architecture for your problem and specify the hyperparameters,
# such as the number of hidden layers and the number of neurons in each layer.

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./checkpoints/model_{epoch}',
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='./logs')
]


def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    model = keras.Sequential([
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation', values=['tanh', 'sigmoid', 'relu']),
            input_shape=(45,)),
    ])
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 4
    # Number of Units: 32 - 512 with stepsize of 32
    num_layers = hp.Int("num_layers", 2, 5)
    for i in range(1, num_layers):
        with hp.conditional_scope('num_layers', list(range(i+1, 5+1))):
            model.add(
                keras.layers.Dense(
                    units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                    activation=hp.Choice('activation_' + str(i), values=['tanh', 'sigmoid', 'relu']),
                )
            )
            # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
            model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))

    # Output layer
    model.add(
        keras.layers.Dense(3, activation='softmax')
    )
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        # optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    model.build()
    return model


In [13]:
# Compile the model: You should compile the model by specifying the optimizer,
# loss function, and evaluation metrics.
import keras_tuner as kt

tuner = kt.tuners.Hyperband(
    build_model,
    objective='val_loss',
    executions_per_trial=2,
    hyperband_iterations=2,
    max_epochs=100,
    factor=3,
    overwrite=False,
    directory='tuner3',
    project_name="hyperband",
)


INFO:tensorflow:Reloading Tuner from tuner3/hyperband/tuner0.json


In [15]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=4, callbacks=[stop_early])


Trial 508 Complete [00h 02m 04s]
val_loss: 0.2693604677915573

Best val_loss So Far: 0.20006748288869858
Total elapsed time: 01h 45m 19s
INFO:tensorflow:Oracle triggered exit


In [14]:

tuner.results_summary()


Results summary
Results in tuner3/hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units: 128
activation: relu
num_layers: 2
units_1: 416
activation_1: sigmoid
dropout_1: 0.2
learning_rate: 0.001
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0228
Score: 0.20006748288869858
Trial summary
Hyperparameters:
units: 480
activation: relu
num_layers: 3
units_1: 320
activation_1: sigmoid
dropout_1: 0.1
learning_rate: 0.001
units_2: 160
activation_2: relu
dropout_2: 0.0
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0203
Score: 0.2034383863210678
Trial summary
Hyperparameters:
units: 384
activation: relu
num_layers: 5
units_1: 448
activation_1: sigmoid
dropout_1: 0.0
learning_rate: 0.001
units_2: 128
activation_2: sigmoid
dropout_2: 0.1
units_3: 192
activation_3: tanh
dropout_3: 0.0
units_4: 320
activation_4: tanh
dropout_4: 0.0
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 3
tuner/ro

In [ ]:
tuner.search_space_summary()


Best val_loss So Far: 0.3011022210121155

- 5 epochs
- 512 units
- tanh activation
- True 2nd layer
- 0.001 learning_rate
- 192 units2


In [17]:
results

,loss,accuracy
Baseline,0.272868,0.896346


In [15]:
params = tuner.get_best_hyperparameters(10)

for i, param in enumerate(params):
    print('Running model:', i, param.values)
    model = tuner.hypermodel.build(param)
    model.fit(x_train, y_train, epochs=param.values.get('tuner/epochs'), callbacks=[stop_early], validation_data=(x_val, y_val), verbose=0)
    hyper_df = evaluate_model(model, x_test, y_test)
    hyper_df.index = ["Hypertuned-" + str(i)]
    hyper_df['params'] = str(param.values)
    # Append results in dataframe
    results = pd.concat([results, hyper_df])

Running model: 0 {'units': 128, 'activation': 'relu', 'num_layers': 2, 'units_1': 416, 'activation_1': 'sigmoid', 'dropout_1': 0.2, 'learning_rate': 0.001, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0228'}
116/116 [==============================] - 0s 1ms/step - loss: 0.2524 - accuracy: 0.9015
Running model: 1 {'units': 480, 'activation': 'relu', 'num_layers': 3, 'units_1': 320, 'activation_1': 'sigmoid', 'dropout_1': 0.1, 'learning_rate': 0.001, 'units_2': 160, 'activation_2': 'relu', 'dropout_2': 0.0, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0203'}
116/116 [==============================] - 0s 1ms/step - loss: 0.2821 - accuracy: 0.8888
Running model: 2 {'units': 384, 'activation': 'relu', 'num_layers': 5, 'units_1': 448, 'activation_1': 'sigmoid', 'dropout_1': 0.0, 'learning_rate': 0.001, 'units_2': 128, 'activation_2': 'sigmoid', 'dropout_2': 0.1, 'units_3': 

Model 1: 116/116 [==============================] - 3s 24ms/step - loss: 0.2007 - accuracy: 0.9369


In [16]:
models = tuner.get_best_models(10)

for i, model in enumerate(models):
    print('Running model:', i+10)
    model.fit(x_train, y_train, epochs=param.values.get('tuner/epochs'), callbacks=[stop_early], validation_data=(x_val, y_val), verbose=0)
    hyper_df = evaluate_model(model, x_test, y_test)
    hyper_df.index = ["Hypertuned-" + str(i+10)]
    # hyper_df['params'] = str(param.values)
    # Append results in dataframe
    results = pd.concat([results, hyper_df])

Running model: 10
116/116 [==============================] - 0s 1ms/step - loss: 0.2011 - accuracy: 0.9269
Running model: 11
116/116 [==============================] - 0s 2ms/step - loss: 0.2703 - accuracy: 0.9323
Running model: 12
116/116 [==============================] - 0s 2ms/step - loss: 0.2503 - accuracy: 0.9304
Running model: 13
116/116 [==============================] - 0s 1ms/step - loss: 0.2304 - accuracy: 0.9215
Running model: 14
116/116 [==============================] - 0s 1ms/step - loss: 0.2098 - accuracy: 0.9296
Running model: 15
116/116 [==============================] - 0s 1ms/step - loss: 0.2077 - accuracy: 0.9250
Running model: 16
116/116 [==============================] - 0s 1ms/step - loss: 0.2248 - accuracy: 0.9218
Running model: 17
116/116 [==============================] - 0s 1ms/step - loss: 0.2197 - accuracy: 0.9307
Running model: 18
116/116 [==============================] - 0s 2ms/step - loss: 0.3176 - accuracy: 0.9004
Running model: 19
116/116 [==========

In [17]:
results

,loss,accuracy,params
Baseline,0.301824,0.884438,NaN
Hypertuned-0,0.252445,0.901488,"{'units': 128, 'activation': 'relu', 'num_laye..."
Hypertuned-1,0.282100,0.888769,"{'units': 480, 'activation': 'relu', 'num_laye..."
Hypertuned-2,0.250474,0.914208,"{'units': 384, 'activation': 'relu', 'num_laye..."
Hypertuned-3,0.259229,0.905819,"{'units': 192, 'activation': 'relu', 'num_laye..."
Hypertuned-4,0.305080,0.897700,"{'units': 320, 'activation': 'sigmoid', 'num_l..."
Hypertuned-5,0.225541,0.916103,"{'units': 128, 'activation': 'relu', 'num_laye..."
Hypertuned-6,0.233592,0.911502,"{'units': 480, 'activation': 'relu', 'num_laye..."
Hypertuned-7,0.227357,0.917997,"{'units': 480, 'activation': 'relu', 'num_laye..."
Hypertuned-8,0.219901,0.934506,"{'units': 480, 'activation': 'relu', 'num_laye..."


In [43]:
best_model_index = int(results['accuracy'].idxmax()['accuracy'].split('-')[1])
best_model_index

8

In [44]:
param = params[best_model_index]


In [51]:
model = build_model(param)
model.fit(x_train, y_train, epochs=param.values.get('tuner/epochs'), callbacks=callbacks, validation_data=(x_val, y_val), verbose=1)

Epoch 1/34


INFO:tensorflow:Assets written to: ./checkpoints/model_1/assets


INFO:tensorflow:Assets written to: ./checkpoints/model_1/assets


347/347 - 4s - loss: 0.6926 - accuracy: 0.7005 - val_loss: 0.5018 - val_accuracy: 0.7932 - 4s/epoch - 10ms/step
Epoch 2/34


INFO:tensorflow:Assets written to: ./checkpoints/model_2/assets


...

INFO:tensorflow:Assets written to: ./checkpoints/model_33/assets


347/347 - 2s - loss: 0.1822 - accuracy: 0.9349 - val_loss: 0.2529 - val_accuracy: 0.9080 - 2s/epoch - 7ms/step
Epoch 34/34


INFO:tensorflow:Assets written to: ./checkpoints/model_34/assets


INFO:tensorflow:Assets written to: ./checkpoints/model_34/assets


347/347 - 2s - loss: 0.1941 - accuracy: 0.9317 - val_loss: 0.2221 - val_accuracy: 0.9185 - 2s/epoch - 7ms/step


In [52]:
evaluate_model(model, x_test, y_test)


116/116 [==============================] - 0s 2ms/step - loss: 0.2189 - accuracy: 0.9194


,loss,accuracy
0,0.218918,0.91935


In [ ]:
r = model.predict(x_test)
# res = []
# tmp = []
# for i in range(24):
#   if i%3 ==0 and i !=0:
#     res.append(tmp)
#     tmp = []
#   tmp.append(r[i])
# res.append(tmp)
# res
res = [np.argmax(i) for i in r]


In [53]:
model.save('./out/model')


INFO:tensorflow:Assets written to: ./out/model/assets


INFO:tensorflow:Assets written to: ./out/model/assets


In [ ]:
# Convert to TFLite model
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(
    './out/model')  # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
